Get subset of FakeNewCorpus data set

In [7]:
import pandas as pd
from cleantext import clean
data = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv')
data['content'] = [clean(entry,
                              lower=True,
                              no_line_breaks=True,
                              no_emails=True,
                              no_urls=True,
                              no_numbers=True,
                               lang="en"
                              ) for entry in data['content']]
data['content'].head()

0    sometimes the power of christmas will make you...
1    awakening of <number> strands of dna - "reconn...
2    never hike alone: a friday the 13th fan film u...
3    when a rare shark was caught, scientists were ...
4    donald trump has the unnerving ability to abil...
Name: content, dtype: object

Using the NLTK module to clean the data. Tokenize, stemm, stop words. Download all packages for ease.

In [1]:
import nltk
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Removing the stopwords:

In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import itertools
data_copy = data.copy(deep=True)

tokens = data_copy['content'].apply(word_tokenize)
filtered_words = [word for word in tokens if word not in stopwords.words('english')]

print(len(list(itertools.chain.from_iterable(filtered_words))))
print(len(list(itertools.chain.from_iterable(data_copy['content']))))

201983
1043166
